In [1]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [2]:
service = Service('/usr/local/bin/chromedriver')

In [3]:
def scroll(driver, timeout, max_scrolls):
    scroll_pause_time = timeout
    
    current_scroll = 0

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             # If heights are the same it will exit the function
#             break
#         last_height = new_height
        
        current_scroll += 1
        if current_scroll >= max_scrolls:
            break

In [4]:
def fetch_wine_links():
    service.start()
    driver = webdriver.Remote(service.service_url)
    driver.get('https://www.vivino.com/explore')
    
    scroll(driver, 2, 10)
    
    page_source = driver.page_source
    
    driver.quit()
    
    return page_source

In [ ]:
vivino_page_source = fetch_wine_links()